In [1]:
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import cv2
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the dataset
dataset_path = r"C:\Users\91939\Downloads\Dog and Cat"
# dataset link : https://www.kaggle.com/datasets/erkamk/cat-and-dog-images-dataset?select=Dog+and+Cat+.png

# Image dimensions
img_height, img_width = 64, 64

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(folder.split('/')[-1])  # Use folder name as label
    return images, labels

# Load images
cat_images, cat_labels = load_images_from_folder(os.path.join(dataset_path, 'Cat'))
dog_images, dog_labels = load_images_from_folder(os.path.join(dataset_path, 'Dog'))

# Combine data
images = cat_images + dog_images
labels = cat_labels + dog_labels

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values
images = images / 255.0

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# Extract HOG features
def extract_hog_features(images):
    hog_features = []
    for image in images:
        feature = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)
        hog_features.append(feature)
    return np.array(hog_features)

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': [1, 0.1, 0.01, 0.001]
}

# Initialize and fit the grid search
grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train_hog, y_train)

# Get the best estimator
best_model = grid.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test_hog)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy after tuning: {accuracy * 100:.2f}%')


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.2s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   0.1s
[CV] END ....................C=0.1, gamma=0.1, 